# Running `fftvis` on Gridded Arrays

<div class="alert alert-warning">

__Warning__

Before running this tutorial, you should make sure you understand the basic concepts and algorithm that `fftvis` uses. You can read up on that here.
</div>

In [1]:
# Standard imports
import numpy as np
import healpy as hp
from astropy.time import Time
import matplotlib.pyplot as plt

# HERA-stack imports
import fftvis
import matvis
from hera_sim.antpos import hex_array
from pyuvdata.telescopes import Telescope
from pyuvdata.analytic_beam import AiryBeam

/home/kartikmandar/anaconda3/envs/matvis-env/lib/python3.11/site-packages/pyuvdata/analytic_beam.py:174: UserWarning: basis_vector_type was not defined, defaulting to azimuth and zenith_angle.
  warnings.warn(


## Mathematical Prelimaries

Simulating radio interferometer visibilities involves calculating the contribution of many sky sources to the signal correlation measured by each baseline pair. For point sources, this is mathematically expressed as a sum:

$$
V(\mathbf{b}) \approx \sum_{j=1}^{N_{\rm sources}} c_j \, e^{-2\pi i \nu \mathbf{b} \cdot \mathbf{x}_j/c}
$$

Evaluating this sum directly is computationally intensive, scaling as $\mathcal{O}(N_{\rm sources} N_{\rm bls})$, which becomes prohibitive for large source catalogs ($N_{\rm sources}$) or many baselines ($N_{\rm bls}$). `fftvis` accelerates this computation significantly by using the non-uniform Fast Fourier Transform (NUFFT), specifically using the high-performance `finufft` library. The exact NUFFT approach and its computational scaling depend on the geometric layout of the array baselines.

For the general case where array baseline vectors $\mathbf{b}_k$ do not form a regular grid (or when considering effects that break perfect regularity, such as errors in antenna placement), `fftvis` treats both the source positions $\mathbf{x}_j$ and the target baseline coordinates $\mathbf{u}_k = (\nu/c)\mathbf{b}_k$ as non-uniform. This "non-uniform to non-uniform" calculation is handled using a **Type 3 NUFFT**. This algorithm internally uses an intermediate uniform grid to perform the bulk of the computation via a standard FFT. The computational cost scales approximately as $\mathcal{O}(N_{\rm sources} w^d + N_{\rm grid} \log N_{\rm grid} + N_{\rm bls} w^d)$. Here, $w$ relates to the desired precision, $d$ is the number of dimensions (2 or 3), and $N_{\rm grid,int}$ is the size of the intermediate grid. Most notably, $N_{\rm grid}$ depends on the product of the maximum sky source extent and the maximum baseline extent (in wavelengths), and can become quite large, especially for wide fields or long baselines, impacting the computational performance.

However, a more efficient approach is possible if the array's baseline vectors $\mathbf{b}_k$ inherently form a regular Cartesian grid (meaning $b_x = k \Delta b_x$, $b_y = l \Delta b_y$, etc., for fixed spacings $\Delta b_x, \Delta b_y$). Since the source positions $\mathbf{x}_j$ remain non-uniform, this becomes a "non-uniform to uniform" problem, ideally suited for a **Type 1 NUFFT**. The Type 1 NUFFT computes visibilities directly onto a uniform output grid covering the extent of the baseline coordinates. Let $N_{\rm unif}$ be the number of points in this target uniform $uv$-grid. The computational scaling for the Type 1 NUFFT is approximately $\mathcal{O}(N_{\rm sources} w^d + N_{\rm unif} \log N_{\rm unif})$.

Comparing the two, the Type 1 NUFFT can be significantly faster than Type 3 if its $N_{\rm unif} \log N_{\rm unif}$ term is considerably smaller than the corresponding $N_{\rm grid} \log N_{\rm grid} + N_{\rm bls} w^d$ terms from Type 3. This advantage occurs when the target uniform grid defined by the baselines ($N_{\rm unif}$) is much smaller than the intermediate grid ($N_{\rm grid}$) that Type 3 requires to handle the general non-uniform case across the same sky and baseline extents. Therefore, for arrays whose baselines naturally form a compact, regular grid, utilizing a Type 1 NUFFT offers a potential performance boost by using this structure. 

In the following cells, we simulate an gridded array using `fftvis` and compare the performance when simulating with a Type 1 and Type 3 NUFFT.

## Setup Telescope / Observation Parameters

First, create our antenna positions. We define this as a dictionary, which maps an antenna number to its 3D East-North-Up position relative to the array centre. To test the difference between the Type 1 and Type 3 NUFFTs as used by `fftvis`, we'll use a hexagonally gridded array layout similar to the layout used by the HERA array.

In [2]:
# define antenna array positions
antpos = hex_array(11, split_core=True, outriggers=2)

Next, we define the beam to be used by all antennas in the array. Unlike `matvis` and `pyuvsim`, `fftvis` currently restricts users to a single beam for all antennas. The specified beam must be a `UVBeam` or `AnalyticBeam` object from `pyuvdata`. Alternatively, you can create a custom `AnalyticBeam` class (see the pyuvdata tutorial on `UVBeam` objects for guidance). For this simulation, we will use a simple, frequency-dependent Airy beam corresponding to a dish size of 14 meters.

In [3]:
# define antenna beam using pyuvdata.analytic_beam.AiryBeam with a dish size of 14 meters
beam = AiryBeam(diameter=14.0)

We also required to provide `fftvis` with the observational configuration including a frequency array, a time array, and a telescope location. The frequency array specifies the observation frequencies in units of Hz. The time array defines the observation times using an `astropy.time.Time` object, with times specified in Julian Dates and configured with the appropriate format and scale. The telescope location specifies the geographic position of the array and can be defined either using `astropy.coordinates.EarthLocation` with a known site name or through `pyuvdata.telescopes.Telescope` by selecting a predefined telescope location supported within `pyuvdata`.

In [4]:
# define a list of frequencies in units of Hz
nfreqs = 2
freqs = np.linspace(100e6, 120e6, nfreqs)

In [5]:
# define a list of times with an astropy time.Time object
ntimes = 3
times = Time(np.linspace(2459845, 2459845.05, ntimes), format='jd', scale='utc')

In [6]:
from astropy.coordinates import EarthLocation

# define using astropy.coordinates.EarthLocation
telescope_loc = EarthLocation.of_site('meerkat')

# define the telescope location using the pyuvdata.telescopes.Telescope
telescope_loc = Telescope.from_known_telescopes('hera').location

## Setup Sky Model

Like `matvis`, `fftvis` makes the point source approximation -- that is it makes breaks a continuous sky model into a discrete number of point sources that it sums over when computing the visibilities. In this notebook, we'll assume the point source approximation by discretizing the sky with a randomly generated HEALpix map.

In [7]:
# number of sources
nside = 64
nsource = hp.nside2npix(nside)

# pixels can be defined as point sources randomly distributed over the full sky
ra = np.deg2rad(np.random.uniform(0, 360, nsource))        # ra of each source (in rad)
dec = np.deg2rad(np.random.uniform(-90, 90.0, nsource))    # dec of each source (in rad)

# define sky model using healpix map
dec, ra = hp.pix2ang(nside, np.arange(nsource))
dec -= np.pi / 2

# define the flux of the sources as a function of frequency. Here, we define smooth spectrum sources
flux = np.random.uniform(0, 1, nsource)                              # flux of each source at 100MHz (in Jy)
alpha = np.ones(nsource) * -0.8                      # sp. index of each source

# Now get the (Nsource, Nfreq) array of the flux of each source at each frequency.
flux_allfreq = ((freqs[:, np.newaxis] / freqs[0]) ** alpha.T * flux.T).T

## Run `fftvis`

Once the simulation parameters are set, the computation is run using the `fftvis.simulate` function. By default, `fftvis` first checks if the provided antenna positions form a regular grid. It does this by using the shortest two non-collinear baseline vectors to define a potential grid basis and then verifying if all antenna positions fit onto that grid. This approach allows `fftvis` to recognize various grid layouts, even those that are skewed or rotated relative to the coordinate axes (like hexagonal arrays, which are essentially skewed regular grids). If a grid is detected, `fftvis` automatically uses the computationally faster Type 1 NUFFT. Since this automatic detection might be unexpected or hide potential configuration issues, the `force_use_type3` parameter is available. Setting this allows a user to enforce the use of the more general Type 3 NUFFT, even if the antenna positions technically form a grid. Additionally, `fftvis` logs a message to inform the user whenever it detects a gridded array configuration and switches to the Type 1 transform.

Below, we perform a simulation of a HERA-like array using both the Type 1 and Type 3 transforms and compare their output.

In [8]:
# Define subset of baselines we're interested in for simulating
baselines = [(i, j) for i in range(len(antpos)) for j in range(len(antpos))]

In [9]:
%%time
# simulate visibilities with the new API
vis_vc_gridded = fftvis.simulate_vis(
    ants=antpos,
    fluxes=flux_allfreq,
    ra=ra,
    dec=dec,
    freqs=freqs,
    times=times.jd,
    telescope_loc=telescope_loc,
    beam=beam,
    polarized=False,
    nprocesses=1,
    baselines=baselines,
    force_use_type3=False, # This is the default value
)

CPU times: user 5.02 s, sys: 324 ms, total: 5.34 s
Wall time: 2.42 s


In [10]:
%%time
# simulate visibilities 
vis_vc_type3 = fftvis.simulate_vis(
    ants=antpos,
    fluxes=flux_allfreq,
    ra=ra,
    dec=dec,
    freqs=freqs,
    times=times.jd,
    telescope_loc=telescope_loc,
    beam=beam,
    polarized=False,
    nprocesses=1,
    baselines=baselines,
    force_use_type3=True, # Even though the antenna layout is gridded we force use type 3
)

CPU times: user 17.2 s, sys: 1.43 s, total: 18.6 s
Wall time: 2.17 s


In [11]:
np.allclose(vis_vc_gridded, vis_vc_type3)

True